<a href="https://colab.research.google.com/github/AhHosny/CIT690E-DL-Course/blob/master/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class convBlock(nn.Module):
  def __init__(self, in_channels, out_channels, **kwargs):
    super(convBlock, self).__init__()

    self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.relu(self.batchnorm(self.conv(x)))

In [ ]:
class inceptionBlock(nn.Module):
  def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
    super(inceptionBlock, self).__init__()

    self.branch1 = convBlock(in_channels, out_1x1, kernel_size=(1,1))

    self.branch2 = nn.Sequential(
        convBlock(in_channels, red_3x3, kernel_size=(1,1)),
        convBlock(red_3x3, out_3x3, kernel_size=(3,3), padding=(1,1)),
    )

    self.branch3 = nn.Sequential(
        convBlock(in_channels, red_5x5, kernel_size=(1,1)),
        convBlock(red_5x5, out_5x5, kernel_size=(5,5), padding=(2,2)),
    )

    self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=(3,3), stride=(1,1), padding=(1,1)),
        convBlock(in_channels, out_1x1pool, kernel_size=(1,1)),
    )

  def forward(self, x):
    # N x filters x 28 x 28
    return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)

In [ ]:
class inceptionAux(nn.Module):
  def __init__(self, in_channels, num_classes):
    super(inceptionAux, self).__init__()

    self.pool = nn.AvgPool2d(kernel_size=5, stride=3)
    self.conv = convBlock(in_channels, 128, kernel_size=1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=0.7)
    self.fc1 = nn.Linear(2048, 1024)
    self.fc2 = nn.Linear(1024, num_classes)

  def forward(self, x):
    x = self.pool(x)
    x = self.conv(x)
    x = x.reshape(x.shape[0], -1)
    x = self.relu(self.fc1(x))
    x = self.fc2(x)

    return x

In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self, num_classes=1000, aux_logits=True):
    super(GoogLeNet, self).__init__()

    assert aux_logits == True or aux_logits == False
    self.aux_logits = aux_logits

    self.conv1 = convBlock(
        in_channels = 3,
        out_channels = 64,
        kernel_size = (7,7),
        stride = (2,2),
        padding = (3,3),
    )

    self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.conv2 = convBlock(64, 192, kernel_size=3, stride=1, padding=1)
    self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    # in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool

    self.inception3a = inceptionBlock(192, 64, 96, 128, 16, 32, 32)
    self.inception3b = inceptionBlock(256, 128, 128, 192, 32, 96, 64)
    self.maxpool3 = nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)

    self.inception4a = inceptionBlock(480, 192, 96, 208, 16, 48, 64)
    self.inception4b = inceptionBlock(512, 160, 112, 224, 24, 64, 64)
    self.inception4c = inceptionBlock(512, 128, 128, 256, 24, 64, 64)
    self.inception4d = inceptionBlock(512, 112, 144, 288, 32, 64, 64)
    self.inception4e = inceptionBlock(528, 256, 160, 320, 32, 128, 128)
    self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.inception5a = inceptionBlock(832, 256, 160, 320, 32, 128, 128)
    self.inception5b = inceptionBlock(832, 384, 192, 384, 48, 128, 128)

    self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
    self.dropout = nn. Dropout(p=0.4)
    self.fc1 = nn.Linear(1024, num_classes)

    if self.aux_logits:
      self.aux1 = inceptionAux(512, num_classes)
      self.aux2 = inceptionAux(528, num_classes)
    else:
      self.aux1 = self.aux2 = None

  def forward(self, x):
    x = self.conv1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)

    x = self.inception3a(x)
    x = self.inception3b(x)
    x = self.maxpool3(x)

    x = self.inception4a(x)

    #aux1

    if self.aux_logits and self.training:
      aux1 = self.aux1(x)

    x = self.inception4b(x)
    x = self.inception4c(x)
    x = self.inception4d(x)

    #aux2
    if self.aux_logits and self.training:
      aux2 = self.aux2(x)

    x = self.inception4e(x)
    x = self.maxpool4(x)
    x = self.inception5a(x)
    x = self.inception5b(x)

    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.dropout(x)
    x = self.fc1(x)

    if self.aux_logits and self.training:
      return aux1, aux2, x
    else:
      return x

In [ ]:
x = torch.randn(3, 3, 224, 224)
model = GoogLeNet(num_classes=1000, aux_logits=True)
print(model)

GoogLeNet(
  (conv1): convBlock(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv2): convBlock(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchnorm): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (inception3a): inceptionBlock(
    (branch1): convBlock(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (branch2): Sequential(
      (0): convBlock(
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
    

In [ ]:
print(model(x)[2].shape)

torch.Size([3, 1000])
